In [2]:
%pip install networkx   # les graphs
%pip install pandas    # les dataframes
%pip install pulp     # programmatoin lineaire

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import networkx as nx

# Saving Tests

In [4]:
import os

# Save the data in a folder
def save_test(folder_path, exams, students, time_slots, rooms):
    if not os.path.exists(folder_path):
        # Create target Directory
        os.mkdir(folder_path)
        print("Directory ", folder_path, " Created ")
    # save all the data
    pd.DataFrame(exams).to_csv(folder_path + "/exams.csv", index=False)
    pd.DataFrame(students).to_csv(folder_path + "/students.csv", index=False)
    pd.DataFrame(time_slots).to_csv(folder_path + "/time_slots.csv", index=False)
    pd.DataFrame(rooms).to_csv(folder_path + "/rooms.csv", index=False)
    print("All files saved in ", folder_path)

## Test 1

In [5]:
# Exemple de données
exams = [
    {"id": 1, "name": "Math", "students": [1, 2], "duration": 1},
    {"id": 2, "name": "Physics", "students": [1, 3], "duration": 2},
    {"id": 3, "name": "Chemistry", "students": [2], "duration": 1},
    {"id": 4, "name": "Biology", "students": [3, 2, 5], "duration": 2}
]

students = [
    {"id": 1, "name": "Justin", "promotion": 1},
    {"id": 2, "name": "Jessi", "promotion": 2},
    {"id": 3, "name": "August", "promotion": 2},
    {"id": 4, "name": "John", "promotion": 1},
    {"id": 5, "name": "Marie", "promotion": 1},
]

time_slots = [
    {"id": 0, "start": "2024-06-01 09:00", "end": "2024-06-01 11:00", "duration": 2},
    {"id": 1, "start": "2024-06-01 13:00", "end": "2024-06-01 15:00", "duration": 2}
]
time_slots.append({"id": 99, "start": "non_programmé", "end": "non_programmé", "duration": float("inf")})

rooms = [
    {"id": 1, "capacity": 5},
    {"id": 2, "capacity": 5},
]

save_test("test1", exams, students, time_slots, rooms)


Directory  test1  Created 
All files saved in  test1


### Test 2

# Welsh Powell avec optimisation avec recherche locale

In [6]:
import networkx as nx

# Construire le graph de conflits entre les examens
# Si deux examens partagent la même arrête elles sont incompatible
def build_conflict_graph(exams):
    G = nx.Graph()
    for exam in exams:
        G.add_node(exam['id'], students=exam['students'], duration=exam['duration'])
    for i, exam1 in enumerate(exams):
        for j, exam2 in enumerate(exams):
            if i < j and set(exam1['students']).intersection(exam2['students']):
                G.add_edge(exam1['id'], exam2['id'])
    return G


In [7]:
# Utiliser l'algorithme de Welsh-Powell pour ressortir une coloration du graphe
def welsh_powell_coloring(G):
    coloring = {}
    nodes = sorted(G.nodes(), key=lambda x: len(list(G.neighbors(x))), reverse=True)
    available_colors = 0
    for node in nodes:
        neighbor_colors = {coloring[neighbor] for neighbor in G.neighbors(node) if neighbor in coloring}
        for color in range(available_colors):
            if color not in neighbor_colors:
                coloring[node] = color
                break
        else:
            coloring[node] = available_colors
            available_colors += 1
    return coloring


In [8]:
# Assigner aux différentes salles les examens en prenant les capacités et les nombres d'étudiants en compte
# A la sortie nous avons une solution initiale réalisable
def assign_rooms_to_exams(coloring, exams, rooms, time_slots):
    schedule = {slot['id']: [] for slot in time_slots}
    penalty_schedule = []
    for exam_id, color in coloring.items():
        exam = next(exam for exam in exams if exam['id'] == exam_id)
        time_slot_id = time_slots[color]['id'] if color < len(time_slots) else 99
        room_assigned = False
        for room in rooms:
            if len(exam['students']) <= room['capacity']:
                schedule[time_slot_id].append({'exam': exam, 'room': room['id']})
                room_assigned = True if time_slot_id != 99 else False
                break
        if not room_assigned:
            penalty_schedule.append(exam_id)
    return schedule, penalty_schedule


In [10]:
# La fonctino de coût que l'on cherche à minimiser
# L'algorithme cherche à minimiser le nombre d'examens non programmés
def calculate_cost(schedule, penalty_schedule, penalty):
    return len(penalty_schedule) * penalty

# Fonction de recherche locale pour améliorer la solution initiale
# Elle essaie de trouver un meilleure solution, compare les coûts et garde la meilleure
def local_search_improvement(schedule, exams, rooms, time_slots, penalty_schedule, penalty):
    best_schedule = schedule.copy()
    best_penalty_schedule = penalty_schedule.copy()
    best_cost = calculate_cost(schedule, penalty_schedule, penalty)
    
    for exam_id in penalty_schedule:
        exam = next(exam for exam in exams if exam['id'] == exam_id)
        for time_slot in time_slots:
            if time_slot['id'] == 99:
                continue
            for room in rooms:
                if room['capacity'] >= len(exam['students']):
                    new_schedule = {k: v.copy() for k, v in best_schedule.items()}
                    new_schedule[time_slot['id']].append({'exam': exam, 'room': room['id']})
                    new_penalty_schedule = [e for e in best_penalty_schedule if e != exam_id]
                    new_cost = calculate_cost(new_schedule, new_penalty_schedule, penalty)
                    # Vérifie si le nouveau coût est plus petit que le meilleur coût
                    if new_cost < best_cost:
                        best_schedule = new_schedule
                        best_penalty_schedule = new_penalty_schedule
                        best_cost = new_cost
    best_penalty_schedule = [sched["exam"]["id"] for sched in schedule.get(99, [])]
    return best_schedule, best_penalty_schedule


### Implémentation

In [11]:
# Construire le graphe des conflits
G = build_conflict_graph(exams)

# Appliquer l'algorithme de Welsh-Powell
coloring = welsh_powell_coloring(G)
print("Coloration: ", coloring, end="\n\n")

# Assigner les salles aux examens
print("Assignation des salles et plages")
schedule, penalty_schedule = assign_rooms_to_exams(coloring, exams, rooms, time_slots)

# Afficher le planning initial
print("\nSchedule : ")
print(*schedule.items(), sep="\n", end="\n\n")
print(penalty_schedule, end="\n\n")

# Améliorer la solution initiale par recherche locale
improved_schedule, improved_penalty_schedule = local_search_improvement(schedule, exams, rooms, time_slots, penalty_schedule, 1000)
print("Improved Schedule : ")
print(*improved_schedule.items(), sep="\n", end="\n\n")


# Afficher le planning amélioré
for time_slot, scheduled_exams in improved_schedule.items():
    if time_slot == 99:
        print(f"Time Slot {time_slot} (non-programmed with penalty):")
        for exam_id in improved_penalty_schedule:
            print(f"  Exam {exams[exam_id - 1]['name']} is not scheduled.")
    else:
        print(f"Time Slot {time_slot}:")
        for item in scheduled_exams:
            exam = item['exam']
            room = item['room']
            print(f"  Exam {exam['name']} in Room {room}")

total_penalty = calculate_cost(improved_schedule, improved_penalty_schedule, 1000)
print(f"Total penalty for non-programmed exams: {total_penalty}")

Coloration:  {1: 0, 4: 1, 2: 2, 3: 2}

Assignation des salles et plages

Schedule : 
(0, [{'exam': {'id': 1, 'name': 'Math', 'students': [1, 2], 'duration': 1}, 'room': 1}])
(1, [{'exam': {'id': 4, 'name': 'Biology', 'students': [3, 2, 5], 'duration': 2}, 'room': 1}])
(99, [{'exam': {'id': 2, 'name': 'Physics', 'students': [1, 3], 'duration': 2}, 'room': 1}, {'exam': {'id': 3, 'name': 'Chemistry', 'students': [2], 'duration': 1}, 'room': 1}])

[2, 3]

Improved Schedule : 
(0, [{'exam': {'id': 1, 'name': 'Math', 'students': [1, 2], 'duration': 1}, 'room': 1}, {'exam': {'id': 2, 'name': 'Physics', 'students': [1, 3], 'duration': 2}, 'room': 1}, {'exam': {'id': 3, 'name': 'Chemistry', 'students': [2], 'duration': 1}, 'room': 1}])
(1, [{'exam': {'id': 4, 'name': 'Biology', 'students': [3, 2, 5], 'duration': 2}, 'room': 1}])
(99, [{'exam': {'id': 2, 'name': 'Physics', 'students': [1, 3], 'duration': 2}, 'room': 1}, {'exam': {'id': 3, 'name': 'Chemistry', 'students': [2], 'duration': 1}, 'ro

# Welsh Powel avec Recherche Taboo

In [12]:
import networkx as nx
import random

# Build the conflict graph
def build_conflict_graph(exams):
    G = nx.Graph()
    for exam in exams:
        G.add_node(exam['id'], students=exam['students'], duration=exam['duration'])
    for i, exam1 in enumerate(exams):
        for j, exam2 in enumerate(exams):
            if i < j and set(exam1['students']).intersection(set(exam2['students'])):
                G.add_edge(exam1['id'], exam2['id'])
    return G

# Apply Welsh-Powell algorithm
def welsh_powell_coloring(G):
    coloring = {}
    nodes = sorted(G.nodes(), key=lambda x: len(list(G.neighbors(x))), reverse=True)
    available_colors = 0
    for node in nodes:
        neighbor_colors = {coloring[neighbor] for neighbor in G.neighbors(node) if neighbor in coloring}
        for color in range(available_colors):
            if color not in neighbor_colors:
                coloring[node] = color
                break
        else:
            coloring[node] = available_colors
            available_colors += 1
    return coloring

# Assign rooms to exams based on initial coloring
def assign_rooms_to_exams(coloring, exams, rooms, time_slots):
    schedule = {slot['id']: [] for slot in time_slots}
    penalty_schedule = []
    for exam_id, color in coloring.items():
        exam = next(exam for exam in exams if exam['id'] == exam_id)
        if color < len(time_slots):
            time_slot_id = time_slots[color]['id']
        else:
            time_slot_id = 99
        room_assigned = False
        for room in rooms:
            if len(exam['students']) <= room['capacity']:
                schedule[time_slot_id].append({'exam': exam, 'room': room['id']})
                room_assigned = True
                break
        if not room_assigned:
            penalty_schedule.append(exam_id)
    return schedule, penalty_schedule

# Cost function
def calculate_cost(schedule, penalty_schedule, penalty):
    return len(penalty_schedule) * penalty

# Tabu Search algorithm
def tabu_search(schedule, exams, rooms, time_slots, penalty_schedule, penalty, max_iterations=100, tabu_tenure=5):
    best_schedule = schedule.copy()
    best_penalty_schedule = penalty_schedule.copy()
    best_cost = calculate_cost(schedule, penalty_schedule, penalty)
    
    tabu_list = []
    
    for iteration in range(max_iterations):
        neighborhood = []

        # Generate neighborhood solutions by trying to reschedule unscheduled exams
        for exam_id in penalty_schedule:
            exam = next(exam for exam in exams if exam['id'] == exam_id)
            for time_slot in time_slots:
                if time_slot['id'] == 99:
                    continue
                for room in rooms:
                    if room['capacity'] >= len(exam['students']):
                        new_schedule = {k: v.copy() for k, v in best_schedule.items()}
                        if time_slot['id'] not in new_schedule:
                            new_schedule[time_slot['id']] = []
                        new_schedule[time_slot['id']].append({'exam': exam, 'room': room['id']})
                        new_penalty_schedule = [e for e in best_penalty_schedule if e != exam_id]
                        new_cost = calculate_cost(new_schedule, new_penalty_schedule, penalty)
                        neighborhood.append((new_schedule, new_penalty_schedule, new_cost, (exam_id, time_slot['id'], room['id'])))
        
        # Select the best neighbor solution not in tabu list
        neighborhood = sorted(neighborhood, key=lambda x: x[2])
        for neighbor in neighborhood:
            if neighbor[3] not in tabu_list:
                best_schedule, best_penalty_schedule, best_cost = neighbor[0], neighbor[1], neighbor[2]
                tabu_list.append(neighbor[3])
                if len(tabu_list) > tabu_tenure:
                    tabu_list.pop(0)
                break
    best_penalty_schedule = [sched["exam"]["id"] for sched in best_schedule.get(99, [])]
    return best_schedule, best_penalty_schedule

# Initial setup
G = build_conflict_graph(exams)
coloring = welsh_powell_coloring(G)
schedule, penalty_schedule = assign_rooms_to_exams(coloring, exams, rooms, time_slots)
print(*schedule.items(), sep="\n", end="\n\n")
# Apply Tabu Search to improve the schedule
improved_schedule, improved_penalty_schedule = tabu_search(schedule, exams, rooms, time_slots, penalty_schedule, 1000)
print(improved_schedule)

# Display the improved schedule
for time_slot, scheduled_exams in improved_schedule.items():
    if time_slot == 99:
        print(f"Time Slot {time_slot} (non-programmed with penalty):")
        for exam_id in improved_penalty_schedule:
            print(f"  Exam {exams[exam_id - 1]['name']} is not scheduled.")
    else:
        print(f"Time Slot {time_slot}:")
        for item in scheduled_exams:
            exam = item['exam']
            room = item['room']
            print(f"  Exam {exam['name']} in Room {room}")

total_penalty = calculate_cost(improved_schedule, improved_penalty_schedule, 1000)
print(f"Total penalty for non-programmed exams: {total_penalty}")

(0, [{'exam': {'id': 1, 'name': 'Math', 'students': [1, 2], 'duration': 1}, 'room': 1}])
(1, [{'exam': {'id': 4, 'name': 'Biology', 'students': [3, 2, 5], 'duration': 2}, 'room': 1}])
(99, [{'exam': {'id': 2, 'name': 'Physics', 'students': [1, 3], 'duration': 2}, 'room': 1}, {'exam': {'id': 3, 'name': 'Chemistry', 'students': [2], 'duration': 1}, 'room': 1}])

{0: [{'exam': {'id': 1, 'name': 'Math', 'students': [1, 2], 'duration': 1}, 'room': 1}], 1: [{'exam': {'id': 4, 'name': 'Biology', 'students': [3, 2, 5], 'duration': 2}, 'room': 1}], 99: [{'exam': {'id': 2, 'name': 'Physics', 'students': [1, 3], 'duration': 2}, 'room': 1}, {'exam': {'id': 3, 'name': 'Chemistry', 'students': [2], 'duration': 1}, 'room': 1}]}
Time Slot 0:
  Exam Math in Room 1
Time Slot 1:
  Exam Biology in Room 1
Time Slot 99 (non-programmed with penalty):
  Exam Physics is not scheduled.
  Exam Chemistry is not scheduled.
Total penalty for non-programmed exams: 2000


# 3. En utilisant la programmation linéaire en nombre entier

Le programme linéaire va optimiser les contraintes molles et s'assurer que les contraintes durs soient remplies.

Une pénalité est rajoutée à la fonction objectif dans le cas où l'algorithme décide de ne pas programmer un examen. Elle le fera donc si elle n'a pas pu converger vers une solution réalisable au programme étant donné les contraintes de non-conflits entre les examens d'un étudiant. 

Le programme linéaire peut être formulé mathématiquement, et sa formulation, bien qu'eshaustive peut être fournise.

In [13]:
# Bibliothèque pour la Programmation Linéaire en nombres entiers
%pip install pulp


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 217.9 kB/s eta 0:01:22
   ---------------------------------------- 0.0/17.7 MB 326.8 kB/s eta 0:00:54
   ---------------------------------------- 0.1/17.7 MB 655.4 kB/s eta 0:00:27
   ---------------------------------------- 0.2/17.7 MB 833.5 kB/s eta 0:00:22
   ---------------------------------------- 0.2/17.7 MB 958.4 kB/s eta 0:00:19
    --------------------------------------- 0.3/17.7 MB 983.0 kB/s eta 0:00:18
    --------------------------------------- 0.3/17.7 MB 983.9 kB/s eta 0:00:18
    --------------------------------------- 0.3/17.7 MB 893.0 kB/s eta 0:00:20
    ------------------------------

In [14]:
import pulp

# Modèle des données

# Examen
exams = [
    {"id": 1, "name": "Math", "students": [1, 2], "duration": 1},
    {"id": 2, "name": "Physics", "students": [1, 3, 4], "duration": 2},
    {"id": 3, "name": "Chemistry", "students": [2], "duration": 1},
    {"id": 4, "name": "Biology", "students": [3, 6, 4, 5], "duration": 2}
]

# Etudiant
students = [
    {"id": 1, "name": "Justin", "promotion": 1},
    {"id": 2, "name": "Jessi", "promotion": 2},
    {"id": 3, "name": "August", "promotion": 2},
    {"id": 4, "name": "John", "promotion": 1},
    {"id": 5, "name": "Marie", "promotion": 1},
    {"id": 6, "name": "Jane", "promotion": 2},
]

# Id des étudiants : L'algorithme n'a pas besoin d'informations personnelles sur les étudiants
students_id = [s["id"] for s in students]

# Plages disponibles
time_slots = [
    {"id": 1, "start": "2024-06-01 09:00", "end": "2024-06-01 11:00", "duration": 2},
    {"id": 2, "start": "2024-06-01 13:00", "end": "2024-06-01 15:00", "duration": 2}
]

# Salle de classe
rooms = [
    {"id": 1, "capacity": 5},
    {"id": 2, "capacity": 5},
]

# Designation : salle-plage
room_slot = dict()
slot_to_time = dict()
idDesignation = 1
for room in rooms:
    for time_period in time_slots:
        room_slot[idDesignation] = (room["id"], time_period["id"])
        slot_to_time[idDesignation] = time_period["id"]
        idDesignation += 1

# Penalty when not programmed
penalty = 1000

# Variables
exam_ids = [exam["id"] for exam in exams]
slot_ids = list(room_slot.keys())
stud_ids = [s["id"] for s in students]
room_ids = [room["id"] for room in rooms]

# Variables de décision du programme linéaire
# Chaque variable associe un examen, une salle et une plage horaire à 1 si l'examen est programmé à cet endroit et 0 sinon
x = pulp.LpVariable.dicts("Examen_dans_la_plage", [(i, k) for i in exam_ids for k in slot_ids], 0, 1, pulp.LpBinary)
x_non_programmed = pulp.LpVariable.dicts("exam_non_programmed", exam_ids, 0, 1, pulp.LpBinary)

# Variables de charge de travail pour chaque étudiant et chaque créneau horaire
W = pulp.LpVariable.dicts("workload", [(sid, t) for sid in stud_ids for t in slot_ids], 0, None, pulp.LpContinuous)
M = pulp.LpVariable.dicts("max_workload", stud_ids, 0, None, pulp.LpContinuous)
m = pulp.LpVariable.dicts("min_workload", stud_ids, 0, None, pulp.LpContinuous)

# Définition du Problème de minimisation
prob = pulp.LpProblem("Programmation_d_examens", pulp.LpMinimize)

# Définiation des contraintes
# Contraintes : chaque examen doit être assigné à un créneau horaire ou à la plage "non_programmée"
for i in exam_ids:
    prob += pulp.lpSum(x[i, k] for k in slot_ids) + x_non_programmed[i] == 1

# Contraintes : pas de conflits pour les étudiants et calcul de la charge pour chaque étudiant
for student in students_id:
    exams_for_student = [exam["id"] for exam in exams if student in exam["students"]]
    for t in time_slots:
        prob += pulp.lpSum(x[i, k] for i in exams_for_student for k in slot_ids if slot_to_time[k] == t["id"]) <= 1  # S'assure qu'il n'y a pas de conflits
        prob += W[student, t["id"]] == pulp.lpSum(x[i, k] for i in exams_for_student for k in slot_ids if slot_to_time[k] == t["id"])  # Calcule la charge de travail pour chaque étudiant et un créneau horaire dans une salle

# Contraintes : capacités des salles
for t in slot_ids:
    room_id = room_slot[t][0]
    room_capacity = next(room for room in rooms if room["id"] == room_id)["capacity"]
    prob += pulp.lpSum(x[i, t] * len(exam["students"]) for i, exam in enumerate(exams, start=1)) <= room_capacity

# Contraintes : Durée des examens
for exam in exams:
    for t in slot_ids:
        time_slot_id = room_slot[t][1]
        duration = next(ts for ts in time_slots if ts["id"] == time_slot_id)["duration"]
        if exam["duration"] > duration:
            prob += x[exam["id"], t] == 0  # Make impossible to program an exam

# Contraintes pour les charges de travail minimales et maximales
for s in students_id:
    for t in time_slots:
        prob += M[s] >= W[s, t["id"]]  # Charge de travail maximale
        prob += m[s] <= W[s, t["id"]]  # Charge de travail minimale

# Fonction objectif : minimiser le déséquilibre total des examens et la pénalisation des examens non programmés
prob += pulp.lpSum(M[s] - m[s] for s in students_id) + penalty * pulp.lpSum(x_non_programmed[i] for i in exam_ids)

# Résolution
prob.solve()

# Affichage des résultats
for i in exam_ids:
    for k in slot_ids:
        if pulp.value(x[i, k]) == 1:
            print(f"Examen {i} est planifié dans la salle {room_slot[k][0]} et le créneau {room_slot[k][1]}")
    if pulp.value(x_non_programmed[i]) == 1:
        print(f"Examen {i} est planifié dans la plage non_programmée avec pénalité")

# Affichage du déséquilibre
for s in students_id:
    print(f"Déséquilibre pour l'étudiant {s}: {pulp.value(M[s]) - pulp.value(m[s])}")


Examen 1 est planifié dans la salle 2 et le créneau 2
Examen 2 est planifié dans la salle 1 et le créneau 1
Examen 3 est planifié dans la salle 2 et le créneau 1
Examen 4 est planifié dans la salle 1 et le créneau 2
Déséquilibre pour l'étudiant 1: 0.0
Déséquilibre pour l'étudiant 2: 0.0
Déséquilibre pour l'étudiant 3: 0.0
Déséquilibre pour l'étudiant 4: 0.0
Déséquilibre pour l'étudiant 5: 1.0
Déséquilibre pour l'étudiant 6: 1.0
